# Libraries

In [1]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
# Driver's path
path = 'C:/Users/papan/Downloads/chromedriver_win32'
driver = webdriver.Chrome(path)
# Maximize Window
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
### Login to dummy linked in account and do the same process because of restriction with authwall login

# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)
# # Accept cookies
driver.find_element('xpath','//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]').click()
# # User Credentials
# Reading txt file where we have our user credentials
with open('user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
user_name = user_credentials[0] # First line
password = user_credentials[1] # Second line
driver.find_element('xpath','//*[@id="username"]').send_keys(user_name)
driver.find_element('xpath','//*[@id="password"]').send_keys(password)
time.sleep(1)
# Login button
driver.find_element('xpath','//*[@id="organic-div"]/form/div[3]/button').click()
time.sleep(10)
driver.implicitly_wait(30)
# if driver.find_element('xpath','//*[@id="ember7"]/h2').text == 'Confirm your account information':
#     driver.find_element('xpath','//*[@id="ember20"]/button[1]').click()
driver.get('https://www.linkedin.com/jobs/search/?currentJobId=3592708886&geoId=104677530&location=Greece')
time.sleep(5)
# driver.execute_script("return window.stop();")
page=1
links = []
print('Links are being collected...')
while True:
    
    try:
        jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.TAG_NAME,'li')
        print(f'Collecting links in page {page}')
        for job in jobs_list:
            if job == jobs_list[-1]:
                all_links = BeautifulSoup(job.get_attribute('innerHTML'),'html').find_all('a',href=True)
                for a in all_links:
                    if a['href'].startswith("/jobs/view") and a['href'] not in links: 
                        links.append(a['href'])
                    else:
                        pass
                page += 1
                driver.find_element('xpath',f'//button[@aria-label="Page {page}"]').click()
                time.sleep(3)
                
            else:
                # scroll down for each job element
                driver.execute_script('arguments[0].scrollIntoView();', job)
                all_links = BeautifulSoup(job.get_attribute('innerHTML'),'html').find_all('a',href=True)
                for a in all_links:
                    if a['href'].startswith("/jobs/view") and a['href'] not in links: 
                        links.append(a['href'])
                    else:
                        pass
    except:
        break

C:\Users\papan\AppData\Local\Temp\ipykernel_16336\1675964536.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Links are being collected...


### prev

In [9]:
# Driver's path
path = 'C:/Users/papan/Downloads/chromedriver_win32'
driver = webdriver.Chrome(path)
# Maximize Window
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
### Login to dummy linked in account and do the same process because of restriction with authwall login

# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)
# # Accept cookies
driver.find_element('xpath','//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]').click()
# # User Credentials
# Reading txt file where we have our user credentials
with open('user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
user_name = user_credentials[0] # First line
password = user_credentials[1] # Second line
driver.find_element('xpath','//*[@id="username"]').send_keys(user_name)
driver.find_element('xpath','//*[@id="password"]').send_keys(password)
time.sleep(1)
# Login button
driver.find_element('xpath','//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)
time.sleep(10)
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = [] 
skills = []
job_desc = []


# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i in range(len(links)):
    try:
        driver.get('https://www.linkedin.com'+links[i])
        time.sleep(2)
        # Click See more.
        buttons = driver.find_elements(By.CLASS_NAME,"artdeco-card__actions")
        for button in buttons:
            if button.text == 'See more':
                button.click()
        time.sleep(2)

    
        # Scraping the job description
        job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')
        for description in job_description:
            job_text = description.find_element(By.CLASS_NAME,"jobs-box__html-content").get_attribute('innerHTML')
            job_desc.append(str(job_text))
            time.sleep(2)  
        
        time.sleep(2)
        # Find the general information of the job offer
        # Used try, except in case there is some missing information for some of the job offers
        contents = driver.find_elements(By.CLASS_NAME, 'p5')
        for content in contents:
            try:
                job_titles.append(content.find_element(By.TAG_NAME, "h1").text)
            except:
                job_titles.append('')  # Append an empty string if element is not found

            try:
                company_names.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__company-name").text)
            except:
                company_names.append('')  # Append an empty string if element is not found

            try:
                company_locations.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").text)
            except:
                company_locations.append('')  # Append an empty string if element is not found

            try:
                work_methods.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__workplace-type").text)
            except:
                work_methods.append('')  # Append an empty string if element is not found

            try:
                post_dates.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__posted-date").text)
            except:
                post_dates.append('')  # Append an empty string if element is not found

            try:
                work_times.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__job-insight").text)
            except:
                work_times.append('')  # Append an empty string if element is not found

            try:
                one_job_skills = ''
                content.find_element(By.CLASS_NAME,'jobs-unified-top-card__job-insight-text-button').click()
                skill_block = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.job-details-skill-match-status-list")))
                one_time_skills = skill_block.find_elements(By.TAG_NAME,'li')
                for skill in one_time_skills:
                    one_job_skills = one_job_skills + skill.text.replace('\nAdd','') + '|'
                skills.append(one_job_skills)
                break
            except:
                skills.append('')
            time.sleep(2)
    
    except:
        pass

    print(f'Scraping the Job Offer {i+1} DONE.')


C:\Users\papan\AppData\Local\Temp\ipykernel_16336\3373032892.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Visiting the links and collecting information just started.
Scraping the Job Offer 1 DONE.
Scraping the Job Offer 2 DONE.
Scraping the Job Offer 3 DONE.
Scraping the Job Offer 4 DONE.
Scraping the Job Offer 5 DONE.
Scraping the Job Offer 6 DONE.
Scraping the Job Offer 7 DONE.
Scraping the Job Offer 8 DONE.
Scraping the Job Offer 9 DONE.
Scraping the Job Offer 10 DONE.
Scraping the Job Offer 11 DONE.
Scraping the Job Offer 12 DONE.
Scraping the Job Offer 13 DONE.
Scraping the Job Offer 14 DONE.
Scraping the Job Offer 15 DONE.
Scraping the Job Offer 16 DONE.
Scraping the Job Offer 17 DONE.
Scraping the Job Offer 18 DONE.
Scraping the Job Offer 19 DONE.
Scraping the Job Offer 20 DONE.
Scraping the Job Offer 21 DONE.
Scraping the Job Offer 22 DONE.
Scraping the Job Offer 23 DONE.
Scraping the Job Offer 24 DONE.
Scraping the Job Offer 25 DONE.
Scraping the Job Offer 26 DONE.
Scraping the Job Offer 27 DONE.
Scraping the Job Offer 28 DONE.
Scraping the Job Offer 29 DONE.
Scraping the Job Offe

### make df

In [34]:
import pandas as pd
Jobs_df = pd.DataFrame({'Title':job_titles,'Company':company_names,'Location':company_locations,'Work Methods':work_methods,
                        'Post Date':post_dates,'Work Times':work_times,'Skills':skills,'Job Description':job_desc})

In [37]:
Jobs_df

,Title,Company,Location,Work Methods,Post Date,Work Times,Skills,Job Description
0,Sous Chef,Casa Cook Hotels,"Samos, Northern Aegean, Greece",On-site,1 week ago,Full-time,Attention to Detail|Communication|Culinary Art...,"\n <h2 class=""text-heading-large mb4"">\..."
1,Cooks A,Santo Collection Oia,"Thíra, Southern Aegean, Greece",On-site,1 week ago,Contract,Boutique Hotels|Cooking|Fine Dining|Hazard Ana...,"\n <h2 class=""text-heading-large mb4"">\..."
2,Χειριστής μηχανών παραγωγής και υπεύθυνος βάρδ...,PALAMIDIS SA,"Athens, Attiki, Greece",On-site,3 weeks ago,Full-time,BRC|ISO Standards|,"\n <h2 class=""text-heading-large mb4"">\..."
3,Σύμβουλος Ομορφιάς,Fresh Line Cosmetics,"Koropi, Attiki, Greece",On-site,2 weeks ago,Full-time,Beauty|Cosmetics|Customer Service|Retail|Sales...,"\n <h2 class=""text-heading-large mb4"">\..."
4,Sr. Manager,Unilever,"Athens, Attiki, Greece",On-site,13 hours ago,Full-time · Mid-Senior level,Communication|Internal Audits|Interpersonal Sk...,"\n <h2 class=""text-heading-large mb4"">\..."
...,...,...,...,...,...,...,...,...
997,Software Engineer in Test,Berkeley Square,Greece,Remote,20 hours ago,Full-time · Mid-Senior level,Apache Kafka|Docker Products|Git|Multithreadin...,"\n <h2 class=""text-heading-large mb4"">\..."
998,Area Representative,Wolt,"Ioannina, Epirus, Greece",Hybrid,1 week ago,Full-time · Mid-Senior level,Account Management|Business Development|Englis...,"\n <h2 class=""text-heading-large mb4"">\..."
999,Junior Consultant - Tax Certificate,Deloitte,"Athens, Attiki, Greece",Hybrid,2 weeks ago,Full-time · Entry level,Accounting|Business|Business Acumen|Business A...,"\n <h2 class=""text-heading-large mb4"">\..."
1000,Salesforce Administrator,Leadfeeder,Greece,Remote,1 week ago,Full-time · Mid-Senior level,Analytical Skills|Certified Salesforce.com Dev...,"\n <h2 class=""text-heading-large mb4"">\..."


In [38]:
#Jobs_df.to_csv('1000_LinkedIn_jobs.csv',encoding='utf-8',index=False,header=True)